In [18]:
import numpy as np
import pandas as pd 

## original data 

In [26]:
nba = pd.read_csv("nba.csv", low_memory = False)
pd.set_option("display.max_column", 30)
pd.set_option("display.max_row", 50)
nba.head(6).T

,0,1,2,3,4,5
Unnamed: 0,0,1,2,3,4,5
game_id,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK
game_date,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17
OT,0,0,0,0,0,0
H_A,A,A,A,A,A,A
...,...,...,...,...,...,...
SG%,36.0,0.0,32.0,10.0,0.0,7.0
SF%,60.0,4.0,67.0,0.0,0.0,62.0
PF%,4.0,85.0,0.0,0.0,0.0,31.0
C%,0.0,11.0,0.0,0.0,100.0,0.0


In [20]:
## first normal form 

nba = nba.drop(["Inactives"],axis = 1)
nba.head(3).T

,0,1,2
Unnamed: 0,0,1,2
game_id,202202170BRK,202202170BRK,202202170BRK
game_date,2022-02-17,2022-02-17,2022-02-17
OT,0,0,0
H_A,A,A,A
...,...,...,...
SG%,36.0,0.0,32.0
SF%,60.0,4.0,67.0
PF%,4.0,85.0,0.0
C%,0.0,11.0,0.0


## Second Normal Form 

In [21]:
# second normal form 
# Only have one column for the primary keep it's kinda cheating
players = nba[["player_id","player"]].drop_duplicates()


In [22]:
games = nba[["game_id","game_date","OT","season"]].drop_duplicates()

In [23]:
# third normal form 

nba = nba.drop(["game_date","OT","season"],axis = 1)


In [27]:
nba.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,...,112108,112109,112110,112111,112112,112113,112114,112115,112116,112117,112118,112119,112120,112121,112122
Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,...,112108,112109,112110,112111,112112,112113,112114,112115,112116,112117,112118,112119,112120,112121,112122
game_id,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,202202170BRK,...,202003030DEN,202003030DEN,202003030DEN,202003030DEN,202003070GSW,202003070GSW,202003070GSW,202003070GSW,202003070GSW,202003070GSW,202003070GSW,202003070GSW,202003070GSW,202003070GSW,202003070GSW
game_date,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,2022-02-17,...,2020-03-03,2020-03-03,2020-03-03,2020-03-03,2020-03-07,2020-03-07,2020-03-07,2020-03-07,2020-03-07,2020-03-07,2020-03-07,2020-03-07,2020-03-07,2020-03-07,2020-03-07
OT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
H_A,A,A,A,A,A,A,A,A,A,A,A,A,A,H,H,...,A,A,A,A,H,H,H,H,H,H,H,H,H,H,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SG%,36.0,0.0,32.0,10.0,0.0,7.0,0.0,0.0,0.0,0.0,38.0,0.0,0.0,31.0,82.0,...,45.0,0.0,44.0,19.0,0.0,0.0,59.0,0.0,65.0,0.0,2.0,45.0,0.0,44.0,19.0
SF%,60.0,4.0,67.0,0.0,0.0,62.0,0.0,33.0,16.0,0.0,0.0,0.0,11.0,58.0,8.0,...,43.0,0.0,48.0,0.0,0.0,14.0,2.0,0.0,16.0,0.0,77.0,43.0,0.0,48.0,0.0
PF%,4.0,85.0,0.0,0.0,0.0,31.0,0.0,57.0,70.0,0.0,0.0,0.0,47.0,11.0,0.0,...,7.0,9.0,8.0,0.0,35.0,76.0,0.0,36.0,0.0,0.0,21.0,7.0,9.0,8.0,0.0
C%,0.0,11.0,0.0,0.0,100.0,0.0,0.0,10.0,14.0,100.0,0.0,100.0,42.0,0.0,0.0,...,0.0,91.0,0.0,0.0,65.0,10.0,0.0,64.0,0.0,0.0,0.0,0.0,91.0,0.0,0.0


In [28]:
nba.columns

Index(['Unnamed: 0', 'game_id', 'game_date', 'OT', 'H_A', 'Team_Abbrev',
       'Team_Score', 'Team_pace', 'Team_efg_pct', 'Team_tov_pct',
       'Team_orb_pct', 'Team_ft_rate', 'Team_off_rtg', 'Inactives',
       'Opponent_Abbrev', 'Opponent_Score', 'Opponent_pace',
       'Opponent_efg_pct', 'Opponent_tov_pct', 'Opponent_orb_pct',
       'Opponent_ft_rate', 'Opponent_off_rtg', 'player', 'player_id',
       'starter', 'mp', 'fg', 'fga', 'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft',
       'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf',
       'pts', 'plus_minus', 'did_not_play', 'is_inactive', 'ts_pct', 'efg_pct',
       'fg3a_per_fga_pct', 'fta_per_fga_pct', 'orb_pct', 'drb_pct', 'trb_pct',
       'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'off_rtg',
       'def_rtg', 'bpm', 'season', 'minutes', 'double_double', 'triple_double',
       'DKP', 'FDP', 'SDP', 'DKP_per_minute', 'FDP_per_minute',
       'SDP_per_minute', 'pf_per_minute', 'ts',
       'last_

## Third Normal Form 

In [29]:
team_game = nba[["game_id","Team_Abbrev",'H_A','Team_Abbrev',
       'Team_Score', 'Team_pace', 'Team_efg_pct', 'Team_tov_pct',
       'Team_orb_pct', 'Team_ft_rate', 'Team_off_rtg',"Opponent_Abbrev"]].drop_duplicates()


In [30]:
team_game

,game_id,Team_Abbrev,H_A,Team_Abbrev,Team_Score,Team_pace,Team_efg_pct,Team_tov_pct,Team_orb_pct,Team_ft_rate,Team_off_rtg,Opponent_Abbrev
0,202202170BRK,WAS,A,WAS,117,94.5,0.627,13.5,22.9,0.157,123.8,BRK
13,202202170BRK,BRK,H,BRK,103,94.5,0.483,13.1,33.3,0.191,109.0,WAS
26,202202170CHO,MIA,A,MIA,111,88.8,0.471,11.1,26.8,0.147,103.4,CHO
37,202202170CHO,CHO,H,CHO,107,88.8,0.453,13.6,28.1,0.221,99.7,MIA
48,202202170LAC,HOU,A,HOU,111,103.7,0.533,15.3,24.0,0.154,107.1,LAC
...,...,...,...,...,...,...,...,...,...,...,...,...
112068,202002270GSW,GSW,H,GSW,86,104.8,0.481,23.6,12.2,0.113,82.1,LAL
112079,202002290PHO,GSW,A,GSW,115,98.6,0.523,9.0,28.9,0.276,116.6,PHO
112090,202003010GSW,GSW,H,GSW,110,100.2,0.522,17.4,38.3,0.191,109.8,WAS
112101,202003030DEN,GSW,A,GSW,116,94.4,0.622,10.7,12.8,0.171,122.9,DEN


In [33]:
player_game = nba.drop(["game_id","Team_Abbrev",'H_A','Team_Abbrev',
       'Team_Score', 'Team_pace', 'Team_efg_pct', 'Team_tov_pct',
       'Team_orb_pct', 'Team_ft_rate', 'Team_off_rtg',"Opponent_Abbrev","Unnamed: 0"],axis = 1)

In [34]:
player_game

,game_date,OT,Inactives,Opponent_Score,Opponent_pace,Opponent_efg_pct,Opponent_tov_pct,Opponent_orb_pct,Opponent_ft_rate,Opponent_off_rtg,player,player_id,starter,mp,fg,...,FDP,SDP,DKP_per_minute,FDP_per_minute,SDP_per_minute,pf_per_minute,ts,last_60_minutes_per_game_starting,last_60_minutes_per_game_bench,PG%,SG%,SF%,PF%,C%,active_position_minutes
0,2022-02-17,0,"Joel Ayayi, Bradley Beal, Kristaps Porzingis",103,94.5,0.483,13.1,33.3,0.191,109.0,Corey Kispert,kispeco01,1,32:30,6,...,22.7,24.75,0.730769230769231,0.698461538461538,0.761538461538461,0.061538,9.00,31.716667,22.017778,1.0,36.0,60.0,4.0,0.0,46.253586
1,2022-02-17,0,"Joel Ayayi, Bradley Beal, Kristaps Porzingis",103,94.5,0.483,13.1,33.3,0.191,109.0,Kyle Kuzma,kuzmaky01,1,30:16,2,...,16.9,17.25,0.652533039647577,0.558370044052863,0.569933920704846,0.099119,7.44,34.324000,18.475954,0.0,0.0,4.0,85.0,11.0,52.152590
2,2022-02-17,0,"Joel Ayayi, Bradley Beal, Kristaps Porzingis",103,94.5,0.483,13.1,33.3,0.191,109.0,Kentavious Caldwell-Pope,caldwke01,1,25:26,3,...,18.0,18.75,0.717562254259502,0.707732634338139,0.737221494102228,0.000000,7.00,29.820290,16.051693,0.0,32.0,67.0,0.0,0.0,47.021807
3,2022-02-17,0,"Joel Ayayi, Bradley Beal, Kristaps Porzingis",103,94.5,0.483,13.1,33.3,0.191,109.0,Raul Neto,netora01,1,20:40,5,...,24.9,25.50,1.13709677419355,1.20483870967742,1.23387096774194,0.048387,7.88,29.920833,14.603922,90.0,10.0,0.0,0.0,0.0,27.603314
4,2022-02-17,0,"Joel Ayayi, Bradley Beal, Kristaps Porzingis",103,94.5,0.483,13.1,33.3,0.191,109.0,Thomas Bryant,bryanth01,1,14:04,5,...,27.0,27.25,1.79502369668246,1.91943127962085,1.93720379146919,0.000000,6.88,20.095833,14.538095,0.0,0.0,0.0,0.0,100.0,36.472537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112118,2020-03-07,0,"Stephen Curry, Draymond Green, Alen Smailagic,...",114,90.9,0.565,10.5,28.2,0.226,125.5,Andrew Wiggins,wiggian01,1,37:04:00,3,...,25.8,26.00,0.660971223,0.696043165,0.701438849,0.107914,13.08,33.110667,19.232562,0.0,2.0,77.0,21.0,0.0,57.207786
112119,2020-03-07,0,"Stephen Curry, Draymond Green, Alen Smailagic,...",114,90.9,0.565,10.5,28.2,0.226,125.5,Juan Toscano-Anderson,toscaju01,1,27:43:00,3,...,16.2,16.50,0.595309681,0.584485869,0.595309681,0.036079,6.00,25.470833,20.228571,5.0,45.0,43.0,7.0,0.0,58.202391
112120,2020-03-07,0,"Stephen Curry, Draymond Green, Alen Smailagic,...",114,90.9,0.565,10.5,28.2,0.226,125.5,Dragan Bender,bendedr01,0,13:15,4,...,15.1,16.25,1.226415094,1.139622642,1.226415094,0.150943,4.00,24.083333,13.228788,0.0,0.0,0.0,9.0,91.0,49.630640
112121,2020-03-07,0,"Stephen Curry, Draymond Green, Alen Smailagic,...",114,90.9,0.565,10.5,28.2,0.226,125.5,Mychal Mulder,muldemy01,1,31:48:00,5,...,23.7,25.25,0.794025157,0.745283019,0.794025157,0.094340,12.64,34.783333,27.691667,0.0,44.0,48.0,8.0,0.0,58.923515


In [35]:
team_game

,game_id,Team_Abbrev,H_A,Team_Abbrev,Team_Score,Team_pace,Team_efg_pct,Team_tov_pct,Team_orb_pct,Team_ft_rate,Team_off_rtg,Opponent_Abbrev
0,202202170BRK,WAS,A,WAS,117,94.5,0.627,13.5,22.9,0.157,123.8,BRK
13,202202170BRK,BRK,H,BRK,103,94.5,0.483,13.1,33.3,0.191,109.0,WAS
26,202202170CHO,MIA,A,MIA,111,88.8,0.471,11.1,26.8,0.147,103.4,CHO
37,202202170CHO,CHO,H,CHO,107,88.8,0.453,13.6,28.1,0.221,99.7,MIA
48,202202170LAC,HOU,A,HOU,111,103.7,0.533,15.3,24.0,0.154,107.1,LAC
...,...,...,...,...,...,...,...,...,...,...,...,...
112068,202002270GSW,GSW,H,GSW,86,104.8,0.481,23.6,12.2,0.113,82.1,LAL
112079,202002290PHO,GSW,A,GSW,115,98.6,0.523,9.0,28.9,0.276,116.6,PHO
112090,202003010GSW,GSW,H,GSW,110,100.2,0.522,17.4,38.3,0.191,109.8,WAS
112101,202003030DEN,GSW,A,GSW,116,94.4,0.622,10.7,12.8,0.171,122.9,DEN


In [36]:
player_game.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112123 entries, 0 to 112122
Data columns (total 70 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   game_date                          112123 non-null  object 
 1   OT                                 112123 non-null  int64  
 2   Inactives                          112010 non-null  object 
 3   Opponent_Score                     112123 non-null  int64  
 4   Opponent_pace                      112123 non-null  float64
 5   Opponent_efg_pct                   112123 non-null  float64
 6   Opponent_tov_pct                   112123 non-null  float64
 7   Opponent_orb_pct                   112123 non-null  float64
 8   Opponent_ft_rate                   112123 non-null  float64
 9   Opponent_off_rtg                   112123 non-null  float64
 10  player                             112123 non-null  object 
 11  player_id                          1121